In [17]:
# 1. Установка micromamba
!wget -qO- https://micromamba.snakepit.net/api/micromamba/linux-64/latest | tar -xvj bin/micromamba

# 2. Создание окружения и установка инструментов
!./bin/micromamba create -y -n bio \
  -c bioconda -c conda-forge \
  fastqc trimmomatic bwa samtools bcftools fastp hisat2 bowtie2 bedtools seqtk multiqc

# 3. Активация окружения и проверка (для примера)
!./bin/micromamba run -n bio fastqc --help
!./bin/micromamba run -n bio bwa
!./bin/micromamba run -n bio samtools --version

bin/micromamba
bioconda/linux-64                                           Using cache
bioconda/noarch                                             Using cache
conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache


Transaction

  Prefix: /root/.local/share/mamba/envs/bio

  Updating specs:

   - fastqc
   - trimmomatic
   - bwa
   - samtools
   - bcftools
   - fastp
   - hisat2
   - bowtie2
   - bedtools
   - seqtk
   - multiqc


  Package                           Version  Build                 Channel           Size
───────────────────────────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────────────────────────

  + _libgcc_mutex                       0.1  conda_forge           conda-forge     Cached
  + _openmp_mutex                       4.5  2_gnu                 conda-forge     Cached
  + alsa-l

# СКАЧИВАЕМ РЕФЕРЕНСНЫЙ ГЕНОМ ( **Pseudomonas aeruginosa**) через NCBI Datasets
datasets accession GCF_000006765.1 --include gff3,rna,cds,protein,genome,seq-report

https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_000006765.1/

In [18]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Pseudomonas_aeruginosa/reference/GCF_000006765.1_ASM676v1/GCF_000006765.1_ASM676v1_genomic.fna.gz

!gunzip GCF_000006765.1_ASM676v1_genomic.fna.gz
!mv GCF_000006765.1_ASM676v1_genomic.fna aeruginosa.fna


--2025-05-09 18:23:13--  https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Pseudomonas_aeruginosa/reference/GCF_000006765.1_ASM676v1/GCF_000006765.1_ASM676v1_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.31, 130.14.250.7, 130.14.250.10, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1787220 (1.7M) [application/x-gzip]
Saving to: ‘GCF_000006765.1_ASM676v1_genomic.fna.gz’

GCF_000006765.1_ASM 100%[===================>]   1.70M  4.18MB/s    in 0.4s    

2025-05-09 18:23:14 (4.18 MB/s) - ‘GCF_000006765.1_ASM676v1_genomic.fna.gz’ saved [1787220/1787220]



# Cкачиваем риды (NGS-данные) для Pseudomonas aeruginosa из NCBI

https://www.ncbi.nlm.nih.gov/sra/SRR12905311[accn]

In [19]:
!apt-get install -y sra-toolkit
!fastq-dump --split-files SRR12905311

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sra-toolkit is already the newest version (2.11.3+dfsg-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Read 6371441 spots for SRR12905311
Written 6371441 spots for SRR12905311


#Проверяем качество ридов триммируем

In [20]:
# Установка FastQC (если ещё не установлен)
!if [ ! -f "/usr/local/bin/fastqc" ]; then \
    wget -O fastqc.zip https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.12.1.zip && \
    unzip -o fastqc.zip && \
    chmod +x FastQC/fastqc && \
    ln -s $(pwd)/FastQC/fastqc /usr/local/bin/fastqc ; \
else echo "FastQC уже установлен"; fi

# Установка fastp v0.23.4
!wget http://opengene.org/fastp/fastp.0.23.4 -O fastp
!chmod +x fastp
!sudo mv fastp /usr/local/bin/

# Проверка версий
!fastqc --version
!fastp --version

# Запуск FastQC
!fastqc SRR12905311_1.fastq SRR12905311_2.fastq

# Тримминг fastp
!fastp \
  -i SRR12905311_1.fastq \
  -I SRR12905311_2.fastq \
  -o clean_1.fastq \
  -O clean_2.fastq \
  -h fastp_report.html \
  -j fastp_report.json


FastQC уже установлен
--2025-05-09 18:27:21--  http://opengene.org/fastp/fastp.0.23.4
Resolving opengene.org (opengene.org)... 8.210.133.117
Connecting to opengene.org (opengene.org)|8.210.133.117|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://opengene.org/fastp/fastp.0.23.4 [following]
--2025-05-09 18:27:22--  https://opengene.org/fastp/fastp.0.23.4
Connecting to opengene.org (opengene.org)|8.210.133.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10946176 (10M) [application/octet-stream]
Saving to: ‘fastp’

fastp               100%[===================>]  10.44M  4.27MB/s    in 2.4s    

2025-05-09 18:27:25 (4.27 MB/s) - ‘fastp’ saved [10946176/10946176]

FastQC v0.12.1
fastp 0.23.4
null
null
Started analysis of SRR12905311_1.fastq
Approx 5% complete for SRR12905311_1.fastq
Approx 10% complete for SRR12905311_1.fastq
Approx 15% complete for SRR12905311_1.fastq
Approx 20% complete for SRR12905311_1.fastq
A

In [21]:
!fastp --version


fastp 0.23.4


In [22]:
#Проверяем создались ли файлы ридов
!ls -lh SRR12905311_*.fastq

-rw-r--r-- 1 root root 1.5G May  9 18:27 SRR12905311_1.fastq
-rw-r--r-- 1 root root 1.5G May  9 18:27 SRR12905311_2.fastq


In [23]:
#Проверяем создались ли очищенные после тримменга файлы ридов
!ls -lh clean_*.fastq

-rw-r--r-- 1 root root 1.4G May  9 18:30 clean_1.fastq
-rw-r--r-- 1 root root 1.4G May  9 18:30 clean_2.fastq


# Выравнивание ридов по генам в формате .bam

In [24]:
#Установка bwa и samtools
!apt-get install -y bwa samtools
#Индексация референсного генома
!bwa index aeruginosa.fna
#Выравнивание ридов dв файл aligned.sam
!bwa mem aeruginosa.fna clean_1.fastq clean_2.fastq > aligned.sam
#Преобразование SAM → BAM → сортировка
!samtools view -S -b aligned.sam > aligned.bam
!samtools sort aligned.bam -o aligned_sorted.bam
!samtools index aligned_sorted.bam


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
bwa is already the newest version (0.7.17-6).
samtools is already the newest version (1.13-4).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
[bwa_index] Pack FASTA... 0.07 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 4.15 seconds elapse.
[bwa_index] Update BWT... 0.04 sec
[bwa_index] Pack forward-only FASTA... 0.03 sec
[bwa_index] Construct SA from BWT and Occ... 1.41 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index aeruginosa.fna
[main] Real time: 5.977 sec; CPU: 5.698 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 125898 sequences (10000018 bp)...
[M::process] read 125904 sequences (10000083 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (210, 52114, 6, 218)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (140, 186, 246)
[M::mem_pes

In [25]:
#Проверка. Покажет количество выровненных ридов, процент покрытия и др
!samtools flagstat aligned_sorted.bam

12485700 + 0 in total (QC-passed reads + QC-failed reads)
12436062 + 0 primary
0 + 0 secondary
49638 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
11184781 + 0 mapped (89.58% : N/A)
11135143 + 0 primary mapped (89.54% : N/A)
12436062 + 0 paired in sequencing
6218031 + 0 read1
6218031 + 0 read2
10600640 + 0 properly paired (85.24% : N/A)
11031604 + 0 with itself and mate mapped
103539 + 0 singletons (0.83% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


Очень высокий процент выравнивания (~90%) — отличное качество данных!

 Более 89% ридов выровнены в парах правильно — тоже отлично.

 Singletons — один рид из пары выровнен, другой — нет. У нас таких всего ~0.83%.

 Данные качественные:

Почти все риды выровнены.

Высокий процент правильно спаренных ридов.

Нет дубликатов или химерных пар.

In [26]:
!samtools view aligned_sorted.bam | head

SRR12905311.108195	163	NC_002516.2	1	60	5S75M	=	395	472	CCCCGTTTAAAGAGACCGGCGATTCTAGTGAAATCGAAAGGGCAGGTCAATTTACAACCAGCGATGACGTAATAGATAGA	AA/A///E/EE6EEEEE/EE/E/EEAEEE/E//EEEAE/AE/EE6EAEE<AEE/EEAEEE/</EEEEE/EEA//EE</AA	NM:i:2	MD:Z:33C14C26	MC:Z:78M	AS:i:65	XS:i:0
SRR12905311.120318	99	NC_002516.2	1	60	1S79M	=	170	248	GTTTAAAGAGACCGGCGATTCTAGTGAAATAGAACGGGCAGGTCAATATCCAACCAGCGATGAAGTAATAGATAGATACA	AAAAAEEEEE6E/E/EEEEEE/EEEEEAEE/EE/EEEAEEE/E6EE//EE/E/EAA/EEEEEE/EA/E///EEAAE/AA/	NM:i:3	MD:Z:29C16T15C16	MC:Z:79M	AS:i:64	XS:i:0
SRR12905311.459357	163	NC_002516.2	1	60	4S76M	=	244	322	CCCGTTTAAAGAGACCGGCGATTCTAGTGAAATCGAACGGGCAGGTCAATTTCCAACCAGCGATGACGTAATAGATAGAT	AAAAAEEEEEEEEEEEEAEEEEEEEEEEAEEAEEEEA/EEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEE<E/EEEAEEE	NM:i:0	MD:Z:76	MC:Z:79M	AS:i:76	XS:i:0
SRR12905311.466260	2209	NC_002516.2	1	60	44H36M	=	120	198	TTTAAAGAGACCGGCGATTCTAGTGAAATCGAACGG	EEEAE/EEE/E<EE/EEEE/EEEEEE/EAEEA/<AA	NM:i:0	MD:Z:36	MC:Z:79M	AS:i:36	XS:i:0	SA:Z:NC_002516.2,6264361,+,44M36S,60,1;
SRR1

In [27]:
# Более подробная статистика (глубина, GC-контент, распределение длин и т.д.).
!samtools stats aligned_sorted.bam > alignment_stats.txt


In [28]:
#Cтатистика по каждому контигу (или хромосоме): число выровненных и невыравненных ридов.
!samtools idxstats aligned_sorted.bam


NC_002516.2	6264404	11184781	103539
*	0	0	1197380


##Провести поиск SNP.

Выбрать 3-4 выявленные мутации. Проверить по NCBI, каким генам они принадлежат

In [29]:
!apt-get update
!apt-get install -y bcftools
# Вызов вариантов с bcftools
!samtools faidx aeruginosa.fna
!bcftools mpileup -f aeruginosa.fna aligned_sorted.bam | bcftools call -mv -Oz -o variants.vcf.gz
!bcftools index variants.vcf.gz

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [41]:

# Установка Java
!apt-get install -y openjdk-21-jdk

# Загрузка и распаковка SnpEff
!wget -O snpEff.zip https://snpeff.blob.core.windows.net/versions/snpEff_latest_core.zip
!unzip -o snpEff.zip
%cd snpEff

# Создание директории для пользовательского генома PAO1
!mkdir -p data/PAO1

# Загрузка FASTA и GFF
!wget -O data/PAO1/sequences.fa.gz https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Pseudomonas_aeruginosa/reference/GCF_000006765.1_ASM676v1/GCF_000006765.1_ASM676v1_genomic.fna.gz
!wget -O data/PAO1/genes.gff.gz     https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Pseudomonas_aeruginosa/reference/GCF_000006765.1_ASM676v1/GCF_000006765.1_ASM676v1_genomic.gff.gz

# Распаковка файлов
!gunzip -f data/PAO1/*.gz

# Редактирование конфигурационного файла
!echo "PAO1.genome : Pseudomonas_aeruginosa_PAO1" >> snpEff.config

# Построение базы данных БЕЗ проверки белков
!java -Xmx4g -jar snpEff.jar build -noCheckProtein -noCheckCds -gff3 -v PAO1

# Аннотация файла variants.vcf.gz (должен быть в /content)
%cd /content/snpEff
!java -Xmx4g -jar snpEff.jar -noDownload -v PAO1 /content/snpEff/variants.vcf.gz > variants.ann.vcf



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-21-jdk is already the newest version (21.0.7+6~us1-0ubuntu1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
--2025-05-09 19:16:39--  https://snpeff.blob.core.windows.net/versions/snpEff_latest_core.zip
Resolving snpeff.blob.core.windows.net (snpeff.blob.core.windows.net)... 20.150.95.164
Connecting to snpeff.blob.core.windows.net (snpeff.blob.core.windows.net)|20.150.95.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66477753 (63M) [application/zip]
Saving to: ‘snpEff.zip’

snpEff.zip          100%[===================>]  63.40M  10.9MB/s    in 5.6s    

2025-05-09 19:16:45 (11.3 MB/s) - ‘snpEff.zip’ saved [66477753/66477753]

Archive:  snpEff.zip
  inflating: snpEff/LICENSE.md       
  inflating: snpEff/snpEff.jar       
  inflating: snpEff/SnpSift.jar      
  inflating: snpEff/galaxy/snpSift_int.xml  
  inflating: snpEff/galaxy/

In [69]:
print('Chromosome      SNP     Gene')
print('_____________________________')
!awk -F '\t' '/^#/ { next } { split($8, info, ";"); for (i in info) if (info[i] ~ /^ANN=/) { split(info[i], ann, "\\|"); print $1"\t"$2"\t"ann[4]; count++; if (count >= 4) exit } }' variants.ann.vcf



Chromosome      SNP     Gene
_____________________________
NC_002516.2	857	dnaA
NC_002516.2	899	dnaA
NC_002516.2	947	dnaA
NC_002516.2	1148	dnaA


SNP находятся в гене dnaA, который кодирует инициатор репликации ДНК — белок DnaA, запускающий процесс репликации хромосомы у бактерий, включая Pseudomonas aeruginosa.

Чем это опасно для бактерии?
Нарушения в DnaA = проблемы с репликацией ДНК

Это может:

замедлить деление

привести к нестабильности генома

сделать бактерию более чувствительной к стрессу или антибиотикам


1	NC_002516.2	Идентификатор хромосомы/контега — это референс последовательность

2	857	Позиция SNP на хромосоме (1-based позиция из VCF)

3	dnaA	Ген, в котором находится данный SNP (получено из поля ANN в VCF)